In [1]:
# First cell - Setup and Imports
import os
import sys
from pathlib import Path
from datetime import datetime, timedelta
from IPython.display import display, HTML
import ipywidgets as widgets
import pandas as pd

# Get the correct project root (DECOHERE directory)
project_root = '/home/siddharth.johri/DECOHERE'
print(f"Project root: {project_root}")  # Debug print

# Add project root to path
sys.path.append(project_root)

# Import PipelineManager
from src.pipeline.pipeline_manager import PipelineManager

# Initialize manager
manager = PipelineManager(project_root)

Project root: /home/siddharth.johri/DECOHERE


In [2]:
# Display system resources
resources = manager.get_system_resources()
display(HTML(f"""
<h3>System Resources</h3>
<ul>
    <li>CPU Cores: {resources['cpu_cores']}</li>
    <li>Available Memory: {resources['available_memory']:.2f} GB</li>
    <li>Memory per Worker: {resources['memory_per_worker']:.2f} GB</li>
    <li>Optimal Workers: {resources['optimal_workers']}</li>
</ul>
"""))

In [3]:
# Create date pickers
start_date = widgets.DatePicker(
    description='Start Date',
    value=datetime(2024, 9, 2),  # Set to September 2, 2024
    disabled=False
)

end_date = widgets.DatePicker(
    description='End Date',
     value=datetime(2024, 9, 2),  # Set to September 2, 2024
    disabled=False
)

num_workers = widgets.IntSlider(
    description='No of Workers',
    value=resources['optimal_workers'],
    min=1,
    max=resources['cpu_cores'] * 2,  # Allow up to logical cores
    step=1
)

# Create a VBox to display widgets vertically
config_widgets = widgets.VBox([
    widgets.HBox([start_date, end_date]),  # Put date pickers side by side
    num_workers
])

# Display the widgets
display(config_widgets)

In [4]:
# Create run button and output area
run_button = widgets.Button(description='Run Pipeline')
output = widgets.Output()

def on_run_button_clicked(b):
    with output:
        print(f"Starting pipeline execution...")
        results_df = manager.run_pipeline(
            start_date.value,
            end_date.value,
            num_workers.value
        )
        
        # Display results
        display(results_df)
        
        # Plot execution times
        manager.plot_execution_times(results_df)

run_button.on_click(on_run_button_clicked)
display(run_button, output)

Button(description='Run Pipeline', style=ButtonStyle())

Output()

In [6]:
# Create check results button and output area
check_button = widgets.Button(description='Check Results')
results_output = widgets.Output()

def on_check_button_clicked(b):
    with results_output:
        results_df = manager.get_latest_results()
        if results_df is not None:
            manager.display_results_summary(results_df)
            display(results_df)
        else:
            print("No results found")

check_button.on_click(on_check_button_clicked)
display(check_button, results_output)

Button(description='Check Results', style=ButtonStyle())

Output()